In [11]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from scipy.signal import welch
import threading


In [12]:
# File paths
eeg_data_path = "/Users/puchku-home/KIIT SEM/PROJECT/EEG/chbmit_preprocessed_data.csv"
seizure_info_path = "/Users/puchku-home/KIIT SEM/PROJECT/EEG/sizuretimes.xlsx" 

In [13]:
# 🔹 Load EEG Data
def load_eeg_data(result_dict):
    if not os.path.exists(eeg_data_path):
        print(f"❌ File not found: {eeg_data_path}")
        result_dict["eeg"] = (None, None)
        return
    
    df = pd.read_csv(eeg_data_path)

    print("📂 EEG Data Loaded. Shape:", df.shape)  # Show dataset size
    
    # Extract time and EEG channel data
    time = np.arange(len(df)) / 256  # Generate time array assuming 256 Hz sampling rate
    eeg_signals = df.iloc[:, :-1]  # Extract all EEG channels (excluding Outcome column)

    result_dict["eeg"] = (time, eeg_signals)

In [14]:
# 🔹 Load Seizure Information
def load_seizure_info(patient_id, result_dict):
    sheet_name = f"chb{str(patient_id).zfill(2)}"
    if not os.path.exists(seizure_info_path):
        print(f"❌ Seizure info file not found: {seizure_info_path}")
        result_dict["seizure"] = None
        return

    try:
        seizure_df = pd.read_excel(seizure_info_path, sheet_name=sheet_name, engine="openpyxl")
        print(f"📊 Seizure info loaded for Patient {patient_id}")
        result_dict["seizure"] = seizure_df
    except Exception as e:
        print(f"⚠ Error loading seizure info for Patient {patient_id}: {e}")
        result_dict["seizure"] = None

In [15]:
# 🔹 Compute and Plot Power Spectral Density (PSD)
def psd_plot(fs, time, eeg_signals, patient_id):
    plt.figure(figsize=(12, 6))
    
    for i in range(eeg_signals.shape[1]):  # Loop through all EEG channels
        f, Pxx = welch(eeg_signals.iloc[:, i], fs=fs, nperseg=1024, window='hann', scaling='density')
        plt.plot(f, Pxx, label=f'Channel {i+1}')  # PSD plot
    
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Power Spectral Density (µV²/Hz)")
    plt.title(f"Power Spectral Density - Patient {patient_id}")
    plt.legend()
    plt.grid()
    plt.show()

In [16]:
# 🔹 Main Function
if __name__ == "__main__":
    fs = 256  # EEG sampling rate
    patient_id = 1  # Example patient

    # Dictionary to store thread results
    result_dict = {}

    # Create threads
    eeg_thread = threading.Thread(target=load_eeg_data, args=(result_dict,))
    seizure_thread = threading.Thread(target=load_seizure_info, args=(patient_id, result_dict))

    # Start threads
    eeg_thread.start()
    seizure_thread.start()

    # Wait for threads to finish
    eeg_thread.join()
    seizure_thread.join()

    # Retrieve results
    time, eeg_signals = result_dict.get("eeg", (None, None))
    seizure_df = result_dict.get("seizure", None)

    if time is not None and eeg_signals is not None and seizure_df is not None:
        psd_plot(fs, time, eeg_signals, patient_id)  # ✅ Compute and plot PSD

⚠ Error loading seizure info for Patient 1: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.
📂 EEG Data Loaded. Shape: (2097150, 24)
